In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [12]:
# Import data
data_path = '/Users/lukecullen/PycharmProjects/petrochemical-data/data/'
estimated_file = data_path+'combined/240325_All/facilityEmissions_all.parquet'

start_year = 2020
end_year = 2050
CCS_gas = 'Carbon dioxide'
gas_nx = 'Nitric oxide' # Actually nitrous oxide, this is to compensate for previous misnamings
CCS_ratio = 0.45
# efficiency = 0.9
summary_gases = ['CO2e_20a', 'CO2e_100a']
process = 'Direct Process'

In [13]:
data = pd.read_parquet(estimated_file)
dp_emissions = data[data['Type']==process]
#deu_emissions = data[data['Type']=='Direct Utilities']

In [14]:
a = 1
b = 0.35
c = 1
M = 15
y = range(0, end_year-start_year+1)

penetration = a / (1+np.exp(-b*(np.array(y)-M)))**a

years = [str(i) for i in range(start_year, end_year+1)]
years_sigma = [year+'_sigma' for year in years]

df = dp_emissions.copy()

In [15]:
def get_gas_reduction(df, CCS_gas, CCS_ratio, penetration, years, years_sigma):
    gas_emissions = df[df['Gas']==CCS_gas]
    new_emissions = gas_emissions[years]*(1-penetration*CCS_ratio)
    new_sigmas = gas_emissions[years_sigma]*(1-penetration*CCS_ratio)
    
    gas_reduction = gas_emissions[years].values - new_emissions.values
    sigma_reduction = gas_emissions[years_sigma].values - new_sigmas.values
    gas_emissions[years] = new_emissions
    gas_emissions[years_sigma] = new_sigmas
    
    return gas_reduction, sigma_reduction, gas_emissions


def get_summary_emissions(df, summary_gas, gas_reduction, sigma_reduction, years, years_sigma, gas_ratio=1):
    """Gas ratio is the ratio of the gas greenhouse effect relative to CO2 for the summary period (20yr or 100yr) desired"""
    summary_emissions = df[df['Gas']==summary_gas]
    summary_emissions[years] = summary_emissions[years].values - gas_reduction*gas_ratio
    summary_emissions[years_sigma] = summary_emissions[years_sigma].values - sigma_reduction*gas_ratio
    return summary_emissions

gas_reduction, sigma_reduction, gas_emissions = get_gas_reduction(df, CCS_gas, CCS_ratio, penetration, years, years_sigma)

# For first gas
ccs_emissions = df[[i not in [CCS_gas]+summary_gases for i in df['Gas']]]

for summary_gas in summary_gases:
    summary_emissions = get_summary_emissions(df, summary_gas, gas_reduction, sigma_reduction, years, years_sigma)
    ccs_emissions = pd.concat((ccs_emissions, summary_emissions))
    
ccs_emissions = pd.concat((ccs_emissions, gas_emissions))

/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_9735/2211383848.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gas_emissions[years] = new_emissions
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_9735/2211383848.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gas_emissions[years_sigma] = new_sigmas
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_9735/2211383848.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [16]:
# For second gas
gas_reduction_nx, sigma_reduction_nx, gas_emissions_nx = get_gas_reduction(ccs_emissions, gas_nx, CCS_ratio, penetration, years, years_sigma)

ccs_emissions_nx = ccs_emissions[[i not in [gas_nx]+summary_gases for i in ccs_emissions['Gas']]]

for summary_gas in summary_gases:
    summary_emissions = get_summary_emissions(ccs_emissions, summary_gas, gas_reduction_nx, sigma_reduction_nx, years, years_sigma, gas_ratio=281)
    ccs_emissions_nx = pd.concat((ccs_emissions_nx, summary_emissions))
    
ccs_emissions_nx = pd.concat((ccs_emissions_nx, gas_emissions_nx))

/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_9735/2211383848.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gas_emissions[years] = new_emissions
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_9735/2211383848.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gas_emissions[years_sigma] = new_sigmas
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_9735/2211383848.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [17]:
#ccs_emissions.to_csv(data_path+'scenarios/dp_ccs_emissions.csv')
ccs_emissions_nx.to_csv(data_path+'scenarios/dp_ccs_emissions_scurve_'+process+'_'+str(CCS_ratio)[2:]+'.csv')